# Implementing ALS using NumPy

In [2]:
import numpy as np

In [100]:
class ALSRecommender:
    def __init__(self, n_factors: int = 10, n_iterations: int = 20, reg_param: float = 0.1):
        """
        Initializes the ALSRecommender.
        
        Parameters:
        - n_factors (int): Number of latent features.
        - n_iterations (int): Number of iterations to perform.
        - reg_param (float): Regularization parameter for ALS.
        """
        self.n_factors = n_factors
        self.n_iterations = n_iterations
        self.reg_param = reg_param
        self.rating_matrix = None
        self.user_factors = None
        self.item_factors = None
        self.n_users = None
        self.n_items = None

    def fit(self, ratings: np.ndarray) -> None:
        """
        Trains the ALS model using the given user-item rating matrix.
        
        Parameters:
        - ratings (np.ndarray): User-item rating matrix with shape (n_users, n_items).
        """ 
        self.rating_matrix = ratings
        self.n_users, self.n_items = ratings.shape
        print(self.n_users, self.n_items)
        self.user_factors = np.random.rand(self.n_users, self.n_factors)
        self.item_factors = np.random.rand(self.n_items, self.n_factors)
        
        for _ in range(self.n_iterations):
            self.user_factors = self.update_users()
            self.item_factors = self.update_items()
    
    def update_items(self) -> np.ndarray:
        """
        Updates items factors matrix.
           
        Returns:
        - np.ndarray: Updated user factors matrix.
        """
        for i in range(len(self.item_factors)):
            rateds = self.rating_matrix[:, i] != 0
            rated_indices = np.where(rateds)[0]
            rated_vectors = self.user_factors[rateds]

            reshaped_vectors = rated_vectors.reshape(-1, self.n_factors, 1)
            ratings_i = self.rating_matrix[rated_indices, i].reshape(-1, 1, 1)

            A = np.sum(reshaped_vectors @ reshaped_vectors.transpose(0, 2, 1), axis=0)
            b = np.sum(reshaped_vectors * ratings_i, axis=0)

            A += np.eye(self.n_factors) * self.reg_param
            A_inv = np.linalg.inv(A)

            self.item_factors[i] = (A_inv @ b).flatten()

        return self.item_factors
    
    def update_users(self) -> np.ndarray:
        """
        Updates user factors matrix.
        
        Returns:
        - np.ndarray: Updated user factors matrix.
        """
        for i in range(len(self.user_factors)):
            rateds = self.rating_matrix[i] != 0
            rated_indices = np.where(rateds)[0]
            rated_vectors = self.item_factors[rateds]

            reshaped_vectors = rated_vectors.reshape(-1, self.n_factors, 1)
            
            ratings_i = self.rating_matrix[i, rated_indices].reshape(-1, 1, 1)

            A = np.sum(reshaped_vectors @ reshaped_vectors.transpose(0, 2, 1), axis=0)
            b = np.sum(reshaped_vectors * ratings_i, axis=0)

            A += np.eye(self.n_factors) * self.reg_param
            A_inv = np.linalg.inv(A)

            self.user_factors[i] = (A_inv @ b).flatten()

        return self.user_factors

    def predict(self) -> np.ndarray:
        """
        Generates the predicted rating matrix.
        
        Returns:
        - np.ndarray: Predicted user-item rating matrix.
        """
        return self.user_factors @ self.item_factors.T
        
    def evaluate(self, test_ratings: np.ndarray) -> float:
        """
        Evaluates the model on test data using RMSE.
        
        Parameters:
        - test_ratings (np.ndarray): User-item rating matrix for testing.
        
        Returns:
        - float: Root Mean Square Error (RMSE).
        """
        predictions = self.predict()
        errors = predictions - test_ratings
        return np.sqrt(np.mean(errors**2))


In [101]:
# TEST
ratings = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [0, 0, 5, 4],
    [0, 3, 4, 0]
])

als = ALSRecommender(n_factors=2, n_iterations=10, reg_param=0.1)

als.fit(ratings)

predicted_ratings = als.predict()
print("Predicted Ratings Matrix:")
print(predicted_ratings)

test_ratings = np.array([
    [5, 3, 2, 1],
    [4, 0, 3, 1],
    [1, 1, 0, 5],
    [2, 0, 5, 4],
    [0, 3, 4, 3]
])

rmse = als.evaluate(test_ratings)
print(f"RMSE on test data: {rmse:.4f}")

5 4
Predicted Ratings Matrix:
[[4.96286338 3.07568264 2.77561099 0.97830112]
 [3.97191717 2.47922219 2.40328685 0.97859287]
 [0.99568411 1.04125696 4.9238098  4.89317439]
 [3.02722473 2.18983905 4.92301039 4.07074124]
 [4.372308   2.86162469 4.00943354 2.54417349]]
RMSE on test data: 1.6819
